# सत्र 2 – RAG मूल्यांकन रागास के साथ

रागास मेट्रिक्स का उपयोग करके न्यूनतम RAG पाइपलाइन का मूल्यांकन करें: उत्तर प्रासंगिकता, सत्यता, संदर्भ सटीकता।


# परिदृश्य
यह परिदृश्य स्थानीय स्तर पर एक न्यूनतम Retrieval Augmented Generation (RAG) पाइपलाइन का मूल्यांकन करता है। हम:
- एक छोटा सिंथेटिक दस्तावेज़ संग्रह परिभाषित करते हैं।
- दस्तावेज़ों को एम्बेड करते हैं और एक साधारण समानता पुनर्प्राप्तकर्ता लागू करते हैं।
- स्थानीय मॉडल (Foundry Local / OpenAI-संगत) का उपयोग करके आधारित उत्तर उत्पन्न करते हैं।
- RAGAS मेट्रिक्स (`answer_relevancy`, `faithfulness`, `context_precision`) की गणना करते हैं।
- एक तेज़ मोड (env `RAG_FAST=1`) का समर्थन करते हैं, जो केवल उत्तर की प्रासंगिकता की गणना करता है ताकि जल्दी से पुनरावृत्ति की जा सके।

इस नोटबुक का उपयोग यह सत्यापित करने के लिए करें कि आपका स्थानीय मॉडल + एम्बेडिंग स्टैक बड़े संग्रह पर स्केल करने से पहले तथ्यात्मक रूप से आधारित उत्तर उत्पन्न करता है।


### व्याख्या: निर्भरता स्थापना
आवश्यक लाइब्रेरी स्थापित करता है:
- `foundry-local-sdk` स्थानीय मॉडल प्रबंधन के लिए।
- `openai` क्लाइंट इंटरफ़ेस के लिए।
- `sentence-transformers` घने एम्बेडिंग के लिए।
- `ragas` + `datasets` मूल्यांकन और मीट्रिक गणना के लिए।
- `langchain-openai` रागास LLM इंटरफ़ेस के लिए एडेप्टर।

पुनः चलाना सुरक्षित है; यदि वातावरण पहले से तैयार है तो इसे छोड़ दें।


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### व्याख्या: मुख्य आयात और मेट्रिक्स  
मुख्य लाइब्रेरी और राग मेट्रिक्स को लोड करता है। मुख्य बिंदु:  
- एम्बेडिंग्स के लिए SentenceTransformer।  
- `evaluate` + चयनित राग मेट्रिक्स।  
- मूल्यांकन कॉर्पस बनाने के लिए `Dataset`।  
ये आयात रिमोट कॉल को ट्रिगर नहीं करते (एम्बेडिंग्स के लिए संभावित मॉडल कैश लोड को छोड़कर)।  


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### व्याख्या: टॉय कॉर्पस और QA ग्राउंड ट्रुथ  
एक छोटे इन-मेमोरी कॉर्पस (`DOCS`), उपयोगकर्ता प्रश्नों का एक सेट, और अपेक्षित ग्राउंड ट्रुथ उत्तरों को परिभाषित करता है। ये बाहरी डेटा को लाने की आवश्यकता के बिना तेज़ और निश्चित मेट्रिक गणना की अनुमति देते हैं। वास्तविक परिदृश्यों में, आप प्रोडक्शन क्वेरी और क्यूरेटेड उत्तरों का नमूना लेंगे।


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### व्याख्या: सेवा प्रारंभ, एम्बेडिंग्स और सुरक्षा पैच  
Foundry Local मैनेजर को प्रारंभ करता है, `promptTemplate` के लिए एक स्कीमा-ड्रिफ्ट सुरक्षा पैच लागू करता है, मॉडल आईडी को हल करता है, OpenAI-संगत क्लाइंट बनाता है, और दस्तावेज़ संग्रह के लिए घनी एम्बेडिंग्स को पहले से गणना करता है। यह पुन: उपयोग योग्य स्थिति को पुनः प्राप्ति + निर्माण के लिए तैयार करता है।


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### व्याख्या: Retriever फ़ंक्शन
एक साधारण वेक्टर समानता रिट्रीवर को परिभाषित करता है जो सामान्यीकृत एम्बेडिंग्स पर डॉट प्रोडक्ट का उपयोग करता है। शीर्ष-k दस्तावेज़ (डिफ़ॉल्ट k=2) लौटाता है। उत्पादन में स्केल और लेटेंसी के लिए इसे ANN इंडेक्स (FAISS, Chroma, Milvus) के साथ बदलें।


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### व्याख्या: जनरेशन फ़ंक्शन
`generate` एक सीमित प्रॉम्प्ट तैयार करता है (सिस्टम निर्देश देता है कि केवल संदर्भ का उपयोग करें) और स्थानीय मॉडल को कॉल करता है। कम तापमान (0.1) सटीक निष्कर्षण को रचनात्मकता पर प्राथमिकता देता है। ट्रिम किए गए उत्तर टेक्स्ट को वापस करता है।


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### व्याख्या: फॉलबैक क्लाइंट प्रारंभ
सुनिश्चित करता है कि `client` मौजूद हो, भले ही पहले का प्रारंभिक सेल छोड़ा गया हो या विफल हो गया हो—बाद के मूल्यांकन चरणों के दौरान NameError को रोकता है।


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### व्याख्या: मूल्यांकन लूप और मीट्रिक्स
मूल्यांकन डेटासेट तैयार करता है (आवश्यक कॉलम: प्रश्न, उत्तर, संदर्भ, ग्राउंड ट्रुथ्स, संदर्भ) और फिर चयनित रागा मीट्रिक्स पर पुनरावृत्ति करता है।

ऑप्टिमाइज़ेशन:
- FAST_MODE उत्तर की प्रासंगिकता तक सीमित रहता है ताकि तेज़ स्मोक टेस्ट किए जा सकें।
- प्रति-मीट्रिक लूप यह सुनिश्चित करता है कि यदि कोई मीट्रिक विफल हो जाए तो पूरी पुनर्गणना से बचा जा सके।

एक डिक्शनरी आउटपुट करता है जिसमें मीट्रिक -> स्कोर (विफलता की स्थिति में NaN) होता है।


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल दस्तावेज़ को उसकी मूल भाषा में आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
